In [1]:
import gzip
import json
import pandas as pd
from my_functions_improved import *

/mount/arbeitsdaten/tcl/Users/dangelo_thesis/thesis/.env/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
/mount/arbeitsdaten/tcl/Users/dangelo_thesis/thesis/.env/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mount/arbeitsdaten/tcl/Users/dangelo_thesis/thesis/.env/lib64/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/mount/arbeitsdaten/tcl/Users/dangelo_thesis/thesis/.env/lib64/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_no

‎𐤀 CLTK version '1.2.3'. When using the CLTK in research, please cite: https://aclanthology.org/2021.acl-demo.3/

Pipeline for language 'Ancient Greek' (ISO: 'grc'): `GreekNormalizeProcess`, `GreekStanzaProcess`, `GreekEmbeddingsProcess`, `StopsProcess`.

⸖ ``LatinSpacyProcess`` using Stanza model by Stanford University from https://stanfordnlp.github.io/stanza/ . Please cite: https://arxiv.org/abs/2003.07082
⸖ ``LatinEmbeddingsProcess`` using word2vec model by University of Oslo from http://vectors.nlpl.eu/ . Please cite: https://aclanthology.org/W17-0237/

⸎ To suppress these messages, instantiate ``NLP()`` with ``suppress_banner=True``.


In [2]:
with gzip.open('noun_dict.json.gz', 'rt', encoding='utf-8') as gzip_file:
    noun_dict = json.load(gzip_file)

In [3]:
with gzip.open('verb_dict.json.gz', 'rt', encoding='utf-8') as gzip_file:
    verb_dict = json.load(gzip_file)

In [4]:
noun_df = pd.DataFrame(noun_dict)

In [5]:
noun_df["form"]

0            ἅβρα
1            ἅβρα
2           ἅβραι
3           ἅβραι
4          ἅβραις
           ...   
210675     ζῳῶδες
210676     ζῳῶδες
210677    ζῳωδίας
210678    ζῳωδίας
210679     ζῳωδίᾳ
Name: form, Length: 210680, dtype: object

In [6]:
l = noun_df.loc[noun_df["form"]=="ἅβρα"]
l

,lemma,form,gender,case,number,dialects
0,ἅβρα,ἅβρα,fem,nom/voc/acc,dual,
1,ἅβρα,ἅβρα,fem,nom/voc,sg,"[(attic, doric, aeolic]"


In [7]:
verb_df = pd.DataFrame(verb_dict)

In [8]:
verb_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
0,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg,
1,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg,
2,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg,
3,ἅλλομαι,ἅλλεσθαι,pres,inf,mid,,,,,
4,ἅλλομαι,ἅλλεσθε,imperf,ind,imperf,,,2nd,pl,"[(doric, aeolic]"
...,...,...,...,...,...,...,...,...,...,...
859327,ζῳοτροφέω,ζῳοτροφεῖν,pres,inf,act,,,,,[doric]
859328,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,masc/neut,gen,,pl,[doric]
859329,ζῳοτροφέω,ζῳοτροφούντων,pres,imperat,imperat,,,3rd,pl,[doric]
859330,ζῳοτροφέω,ζῳοτροφοῦσιν,pres,part,act,masc/neut,dat,,pl,[doric]


In [9]:
strings = ["ἂν δὲ καὶ αὐτοὶ βάντες ἐπὶ κληῖσι καθῖζον", "ἑξῆς δ᾽ ἑζόμενοι πολιὴν ἅλα τύπτον ἐρετμοῖς", "ἔνθεν δὲ προτέρω πλέομεν ἀκαχήμενοι ἦτορ", "ἄσμενοι ἐκ θανάτοιο, φίλους ὀλέσαντες ἑταίρους", "νῆα μὲν οἵ γε μέλαιναν ἁλὸς βένθοσδε ἔρυσσαν", "τοὶ δὲ πρυμνήσι᾽ ἔλυσαν"]

In [10]:
def get_noun_analysis(noun, df):
    noun_analysis = []
    word_df = df.loc[df["form"]==noun]
    for id, row in word_df.iterrows():
        noun_analysis.append([row["gender"], row["case"], row["number"], row ["dialects"]])
    return noun_analysis

In [11]:
def get_verb_analysis(verb, df):
    verb_analysis = []
    word_df = df.loc[df["form"]==verb]
    for id, row in word_df.iterrows():
        verb_analysis.append([row["tense"], row["mode"], row["act/mid/p"], row ["gender"], row["case"], row["person"], row["number"], row["dialects"]])
    return verb_analysis

In [14]:
import pickle

In [16]:
with open("sents_sim_dict.pickle", "rb") as file:
    sents_sim = pickle.load(file)

In [33]:
def get_lemma_syn(word, sim_dic):
    lemma_w = lemmatize_cltk(word)[0]
    
    for sentence in sim_dic:
        for lemma in sim_dic[sentence]:
            if lemma == lemma_w:
                print("lol")
                sorted_values = sorted(sim_dic[sentence][lemma], key=lambda x: x[1], reverse=True)
                syn, value = sorted_values[1]
                return syn

In [34]:
s = get_lemma_syn("βάντες", sents_sim)
s

lol


'διαβαίνω'

In [13]:
s = "ἂν δὲ καὶ αὐτοὶ βάντες ἐπὶ κληῖσι καθῖζον"
sent_analysis = {}
sent_split = s.split(" ")
for id, word in enumerate(sent_split):
    word_analysis = []
    pos = pos_tag_grecy(word)[0]
    if pos == "a" or pos == "n":
        word_analysis = get_noun_analysis(word, noun_df)
    if pos == "v":
        word_analysis = get_verb_analysis(word, verb_df)
    if word_analysis != []:
        for analysis in word_analysis:
            lemma_synonym = get_lemma_syn(word)
            inflected_syn = get_infl_syn(lemma_synonym)
            sent_split[id] = inflected_syn


sent_analysis
            

NameError: name 'get_lemma_syn' is not defined